# CBP
Retrieves data from the Census Bureau's County Business Patterns API.

In [1]:
# https://www.census.gov/programs-surveys/cbp/data.html
# Importing necessary packages.
import zipfile as zip
import pandas as pd
import csv
import os
import pyarrow.feather as feather

pd.set_option('display.max_columns', None)

# Set up file path:
from google.colab import drive
drive.mount('/content/gdrive')
%cd "/content/gdrive/My Drive/census-api"
target = os.getcwd()

import requests, sqlite3, json
from IPython.display import clear_output

Mounted at /content/gdrive
/content/gdrive/My Drive/census-api


In [2]:
os.getcwd()
base_url = f'https://api.census.gov/data/'
ejsonpath=os.path.join(target,'emp_data.json')
ecols='EMP,PAYANN'

## API Call and Processing
Request the CBP data from the Census Bureau

### CBP Employment Data by Industry
This data can be requested in a single api call - do not rerun the requests block if retrieval is successful but subsequent notebook changes are needed. Proceed to the next block and pull data from json dump file.

#### ***THIS BLOCK IS A REQUESTS BLOCK!*** 

### Different years use different industry code 
NAICS2017 #NAICS2012 #NAICS2007 #NAICS2002 #NAICS1997 #SIC

API avaliable data list:
https://api.census.gov/data.html

NAICS:
https://www.census.gov/naics/ 


###2017 to 2020 use NAICS2017

In [3]:

for year in range(2017,2021):
  print(year)
  #Code 200 = success, do not rerun this block unless it's necessary
  edata_url = f'{base_url}{year}/cbp?get={ecols}&for=county:*&NAICS2017=31-33'
  print(edata_url)
  response=requests.get(edata_url)
  if response.status_code==200:
      emp_data=response.json()
      with open(ejsonpath, 'w') as f:
          json.dump(emp_data, f)
      print('Data dumped to json file')
  else:
      print('Problem with retrieval, response code',response.status_code)
      break
  with open(ejsonpath, 'r') as f:
      ejsondata=json.load(f)
  cbpemp=pd.DataFrame(ejsondata[1:], columns=ejsondata[0])
  cbpemp['YEAR'] = year
  if year==2017:  # change the start year!
    df=cbpemp
  else:
    df=df.append(cbpemp)

2017
https://api.census.gov/data/2017/cbp?get=EMP,PAYANN&for=county:*&NAICS2017=31-33
Data dumped to json file
2018
https://api.census.gov/data/2018/cbp?get=EMP,PAYANN&for=county:*&NAICS2017=31-33
Data dumped to json file
2019
https://api.census.gov/data/2019/cbp?get=EMP,PAYANN&for=county:*&NAICS2017=31-33
Data dumped to json file
2020
https://api.census.gov/data/2020/cbp?get=EMP,PAYANN&for=county:*&NAICS2017=31-33
Data dumped to json file


In [4]:
df_2017_2020 = df
df_2017_2020.sort_values(by=['state','county','YEAR'], inplace=True)
display(df_2017_2020)
df_2017_2020.to_stata('cbp_manupayroll2017_2020.dta', write_index=False)  

,EMP,PAYANN,NAICS2017,state,county,YEAR
1,920,75622,31-33,01,001,2017
2635,1042,80452,31-33,01,001,2018
294,1185,81009,31-33,01,001,2019
2625,949,77500,31-33,01,001,2020
2,4160,209675,31-33,01,003,2017
...,...,...,...,...,...,...
1515,19,798,31-33,78,020,2019
2968,169,6950,31-33,78,030,2017
923,167,6822,31-33,78,030,2018
1516,189,7264,31-33,78,030,2019


###2012 to 2016 use NAICS2012 code

In [5]:

for year in range(2012,2017):
  print(year)
  #Code 200 = success, do not rerun this block unless it's necessary
  edata_url = f'{base_url}{year}/cbp?get={ecols}&for=county:*&NAICS2012=31-33'
  print(edata_url)
  response=requests.get(edata_url)
  if response.status_code==200:
      emp_data=response.json()
      with open(ejsonpath, 'w') as f:
          json.dump(emp_data, f)
      print('Data dumped to json file')
  else:
      print('Problem with retrieval, response code',response.status_code)
      break
  with open(ejsonpath, 'r') as f:
      ejsondata=json.load(f)
  cbpemp=pd.DataFrame(ejsondata[1:], columns=ejsondata[0])
  cbpemp['YEAR'] = year
  if year==2012:
    df=cbpemp
  else:
    df=df.append(cbpemp)

2012
https://api.census.gov/data/2012/cbp?get=EMP,PAYANN&for=county:*&NAICS2012=31-33
Data dumped to json file
2013
https://api.census.gov/data/2013/cbp?get=EMP,PAYANN&for=county:*&NAICS2012=31-33
Data dumped to json file
2014
https://api.census.gov/data/2014/cbp?get=EMP,PAYANN&for=county:*&NAICS2012=31-33
Data dumped to json file
2015
https://api.census.gov/data/2015/cbp?get=EMP,PAYANN&for=county:*&NAICS2012=31-33
Data dumped to json file
2016
https://api.census.gov/data/2016/cbp?get=EMP,PAYANN&for=county:*&NAICS2012=31-33
Data dumped to json file


In [6]:
df_2012_2016 = df
df_2012_2016.sort_values(by=['state','county','YEAR'], inplace=True)
display(df_2012_2016)
df_2012_2016.to_stata('cbp_manupayroll2012_2016.dta', write_index=False) 

,EMP,PAYANN,NAICS2012,state,county,YEAR
2754,0,0,31-33,01,001,2012
2564,907,55932,31-33,01,001,2013
2751,0,0,31-33,01,001,2014
1,971,64263,31-33,01,001,2015
2751,984,65080,31-33,01,001,2016
...,...,...,...,...,...,...
1368,224,10897,31-33,78,030,2012
1182,205,10770,31-33,78,030,2013
1365,181,8317,31-33,78,030,2014
3163,190,8788,31-33,78,030,2015


###2008 to 2011 use NAICS2007

In [7]:

for year in range(2008,2012):
  print(year)
  #Code 200 = success, do not rerun this block unless it's necessary
  edata_url = f'{base_url}{year}/cbp?get={ecols}&for=county:*&NAICS2007=31-33'
  print(edata_url)
  response=requests.get(edata_url)
  if response.status_code==200:
      emp_data=response.json()
      with open(ejsonpath, 'w') as f:
          json.dump(emp_data, f)
      print('Data dumped to json file')
  else:
      print('Problem with retrieval, response code',response.status_code)
      break
  with open(ejsonpath, 'r') as f:
      ejsondata=json.load(f)
  cbpemp=pd.DataFrame(ejsondata[1:], columns=ejsondata[0])
  cbpemp['YEAR'] = year
  if year==2008:
    df=cbpemp
  else:
    df=df.append(cbpemp)

2008
https://api.census.gov/data/2008/cbp?get=EMP,PAYANN&for=county:*&NAICS2007=31-33
Data dumped to json file
2009
https://api.census.gov/data/2009/cbp?get=EMP,PAYANN&for=county:*&NAICS2007=31-33
Data dumped to json file
2010
https://api.census.gov/data/2010/cbp?get=EMP,PAYANN&for=county:*&NAICS2007=31-33
Data dumped to json file
2011
https://api.census.gov/data/2011/cbp?get=EMP,PAYANN&for=county:*&NAICS2007=31-33
Data dumped to json file


In [8]:
df_2008_2012 = df
df_2008_2012.sort_values(by=['state','county','YEAR'], inplace=True)
display(df_2008_2012)
df_2008_2012.to_stata('cbp_manupayroll2008_2012.dta', write_index=False) 

,EMP,PAYANN,NAICS2007,state,county,YEAR
152,0,0,31-33,01,001,2008
0,0,0,31-33,01,001,2009
8,0,0,31-33,01,001,2010
6,0,0,31-33,01,001,2011
151,4923,163366,31-33,01,003,2008
...,...,...,...,...,...,...
3158,0,234,31-33,78,020,2011
150,402,14481,31-33,78,030,2008
3158,237,8478,31-33,78,030,2009
3156,206,7425,31-33,78,030,2010


### 2003 to 2007 use the NAICS2002 code

In [9]:

for year in range(2003,2008):
  print(year)
  #Code 200 = success, do not rerun this block unless it's necessary
  edata_url = f'{base_url}{year}/cbp?get={ecols}&for=county:*&NAICS2002=31-33'
  print(edata_url)
  response=requests.get(edata_url)
  if response.status_code==200:
      emp_data=response.json()
      with open(ejsonpath, 'w') as f:
          json.dump(emp_data, f)
      print('Data dumped to json file')
  else:
      print('Problem with retrieval, response code',response.status_code)
      break
  with open(ejsonpath, 'r') as f:
      ejsondata=json.load(f)
  cbpemp=pd.DataFrame(ejsondata[1:], columns=ejsondata[0])
  cbpemp['YEAR'] = year
  if year==2003:
    df=cbpemp
  else:
    df=df.append(cbpemp)

2003
https://api.census.gov/data/2003/cbp?get=EMP,PAYANN&for=county:*&NAICS2002=31-33
Data dumped to json file
2004
https://api.census.gov/data/2004/cbp?get=EMP,PAYANN&for=county:*&NAICS2002=31-33
Data dumped to json file
2005
https://api.census.gov/data/2005/cbp?get=EMP,PAYANN&for=county:*&NAICS2002=31-33
Data dumped to json file
2006
https://api.census.gov/data/2006/cbp?get=EMP,PAYANN&for=county:*&NAICS2002=31-33
Data dumped to json file
2007
https://api.census.gov/data/2007/cbp?get=EMP,PAYANN&for=county:*&NAICS2002=31-33
Data dumped to json file


In [10]:
df_2003_2007 = df
df_2003_2007.sort_values(by=['state','county','YEAR'], inplace=True)
display(df_2003_2007)
df_2003_2007.to_stata('cbp_manupayroll2003_2007.dta', write_index=False) 

,EMP,PAYANN,NAICS2002,state,county,YEAR
830,1957,80340,31-33,01,001,2003
850,1822,77738,31-33,01,001,2004
757,1623,73871,31-33,01,001,2005
761,1565,76206,31-33,01,001,2006
561,0,0,31-33,01,001,2007
...,...,...,...,...,...,...
740,0,0,31-33,72,999,2003
842,0,0,31-33,72,999,2004
843,0,0,31-33,72,999,2004
730,0,0,31-33,72,999,2005


### 1998 to 2002 use the NAICS1997 code

In [11]:

for year in range(1998,2003):
  print(year)
  #Code 200 = success, do not rerun this block unless it's necessary
  edata_url = f'{base_url}{year}/cbp?get={ecols}&for=county:*&NAICS1997=31-33'
  print(edata_url)
  response=requests.get(edata_url)
  if response.status_code==200:
      emp_data=response.json()
      with open(ejsonpath, 'w') as f:
          json.dump(emp_data, f)
      print('Data dumped to json file')
  else:
      print('Problem with retrieval, response code',response.status_code)
      break
  with open(ejsonpath, 'r') as f:
      ejsondata=json.load(f)
  cbpemp=pd.DataFrame(ejsondata[1:], columns=ejsondata[0])
  cbpemp['YEAR'] = year
  if year==1998:
    df=cbpemp
  else:
    df=df.append(cbpemp)

1998
https://api.census.gov/data/1998/cbp?get=EMP,PAYANN&for=county:*&NAICS1997=31-33
Data dumped to json file
1999
https://api.census.gov/data/1999/cbp?get=EMP,PAYANN&for=county:*&NAICS1997=31-33
Data dumped to json file
2000
https://api.census.gov/data/2000/cbp?get=EMP,PAYANN&for=county:*&NAICS1997=31-33
Data dumped to json file
2001
https://api.census.gov/data/2001/cbp?get=EMP,PAYANN&for=county:*&NAICS1997=31-33
Data dumped to json file
2002
https://api.census.gov/data/2002/cbp?get=EMP,PAYANN&for=county:*&NAICS1997=31-33
Data dumped to json file


In [12]:
df_1998_2002 = df
df_1998_2002.sort_values(by=['state','county','YEAR'], inplace=True)
display(df_1998_2002)
df_1998_2002.to_stata('cbp_manupayroll1998_2002.dta', write_index=False) 

,EMP,PAYANN,NAICS1997,state,county,YEAR
0,1803,68867,31-33,01,001,1998
2080,2048,77052,31-33,01,001,1999
2054,2058,74903,31-33,01,001,2000
2065,1943,68137,31-33,01,001,2001
1980,1861,70787,31-33,01,001,2002
...,...,...,...,...,...,...
3079,0,0,31-33,56,045,1998
2066,0,0,31-33,56,045,1999
2036,0,0,31-33,56,045,2000
2053,0,0,31-33,56,045,2001


### 1986 to 1997 use SIC code

SIC code description (1986-1987):

https://www2.census.gov/programs-surveys/cbp/technical-documentation/records-layouts/sic-code-descriptions/sic86_87.txt

SIC code description (1988-1997):

https://www2.census.gov/programs-surveys/cbp/technical-documentation/records-layouts/sic-code-descriptions/sic88_97.txt

County Business Patterns: 1986

"It was incorrectly coded to "19--", it has been updated to the correct sector code of "20--". "from https://www.census.gov/data/datasets/1986/econ/cbp/1986-cpb.html

However, we can only get data from code "19--" from the API request.

In [13]:

for year in range(1986,1987):
  print(year)
  #Code 200 = success, do not rerun this block unless it's necessary
  edata_url = f'{base_url}{year}/cbp?get={ecols}&for=county:*&SIC=19'
  print(edata_url)
  response=requests.get(edata_url)
  if response.status_code==200:
      emp_data=response.json()
      with open(ejsonpath, 'w') as f:
          json.dump(emp_data, f)
      print('Data dumped to json file')
  else:
      print('Problem with retrieval, response code',response.status_code)
      break
  with open(ejsonpath, 'r') as f:
      ejsondata=json.load(f)
  cbpemp=pd.DataFrame(ejsondata[1:], columns=ejsondata[0])
  cbpemp['YEAR'] = year
  if year==1986:
    df=cbpemp
  else:
    df=df.append(cbpemp)

1986
https://api.census.gov/data/1986/cbp?get=EMP,PAYANN&for=county:*&SIC=19
Data dumped to json file


In [14]:
df_1986 = df
df_1986.sort_values(by=['state','county','YEAR'], inplace=True)
display(df_1986)
df_1986.to_stata('cbp_manupayroll1986.dta', write_index=False) 

,EMP,PAYANN,SIC,state,county,YEAR
0,1633,40334,19,01,001,1986
1,4557,73666,19,01,003,1986
2,2768,44033,19,01,005,1986
3,1740,20668,19,01,007,1986
4,1093,13173,19,01,009,1986
...,...,...,...,...,...,...
3106,346,10818,19,56,037,1986
3107,183,3545,19,56,039,1986
3108,153,3598,19,56,041,1986
3109,454,12504,19,56,043,1986


In [15]:
for year in range(1987,1998):
  print(year)
  #Code 200 = success, do not rerun this block unless it's necessary
  edata_url = f'{base_url}{year}/cbp?get={ecols}&for=county:*&SIC=20'
  print(edata_url)
  response=requests.get(edata_url)
  if response.status_code==200:
      emp_data=response.json()
      with open(ejsonpath, 'w') as f:
          json.dump(emp_data, f)
      print('Data dumped to json file')
  else:
      print('Problem with retrieval, response code',response.status_code)
      break
  with open(ejsonpath, 'r') as f:
      ejsondata=json.load(f)
  cbpemp=pd.DataFrame(ejsondata[1:], columns=ejsondata[0])
  cbpemp['YEAR'] = year
  if year==1987:
    df=cbpemp
  else:
    df=df.append(cbpemp)

1987
https://api.census.gov/data/1987/cbp?get=EMP,PAYANN&for=county:*&SIC=20
Data dumped to json file
1988
https://api.census.gov/data/1988/cbp?get=EMP,PAYANN&for=county:*&SIC=20
Data dumped to json file
1989
https://api.census.gov/data/1989/cbp?get=EMP,PAYANN&for=county:*&SIC=20
Data dumped to json file
1990
https://api.census.gov/data/1990/cbp?get=EMP,PAYANN&for=county:*&SIC=20
Data dumped to json file
1991
https://api.census.gov/data/1991/cbp?get=EMP,PAYANN&for=county:*&SIC=20
Data dumped to json file
1992
https://api.census.gov/data/1992/cbp?get=EMP,PAYANN&for=county:*&SIC=20
Data dumped to json file
1993
https://api.census.gov/data/1993/cbp?get=EMP,PAYANN&for=county:*&SIC=20
Data dumped to json file
1994
https://api.census.gov/data/1994/cbp?get=EMP,PAYANN&for=county:*&SIC=20
Data dumped to json file
1995
https://api.census.gov/data/1995/cbp?get=EMP,PAYANN&for=county:*&SIC=20
Data dumped to json file
1996
https://api.census.gov/data/1996/cbp?get=EMP,PAYANN&for=county:*&SIC=20
Data 

In [16]:
df_1987_1997 = df
df_1987_1997.sort_values(by=['state','county','YEAR'], inplace=True)
display(df_1987_1997)
df_1987_1997.to_stata('cbp_manupayroll1987_1997.dta', write_index=False) 

,EMP,PAYANN,SIC,state,county,YEAR
3,1932,50663,20,01,001,1987
0,2435,58307,20,01,001,1988
0,2492,56610,20,01,001,1989
0,2321,57179,20,01,001,1990
1,2376,63072,20,01,001,1992
...,...,...,...,...,...,...
307,8354,266224,20,9,15,1995
303,75606,3253344,20,9,3,1995
302,17451,617304,20,9,5,1995
304,12910,462457,20,9,7,1995
